# Classifying Asteroids as Hazardous

The data is about Asteroids - NeoWs.

NeoWs (Near Earth Object Web Service) is a RESTful web service for near earth Asteroid information. With NeoWs a user can: search for Asteroids based on their closest approach date to Earth, lookup a specific Asteroid with its NASA JPL small body id, as well as browse the overall data-set.

The goal of this project is to find potential hazardous and non-hazardous asteroids taking into account the features responsible for qualifying an asteroid as hazardous.

In [3]:
# Imports
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn import neighbors
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

## Data Preprocess

The dataset contains 4687 entries and provides information on numerous asteroids. Each asteroid is classified as either hazardous or non-hazardous, depending on the value of the ***Hazardous*** column being either ```True``` or ```False```.

The table below showcases the original attributes of each entry of the dataset:

| Attribute                       | Type    |
|---------------------------------|----------|
| Neo Reference ID                | int64    |
| Name                            | int64    |
| Absolute Magnitude              | float64  |
| Est Dia in KM (min)             | float64  |
| Est Dia in KM (max)             | float64  |
| Est Dia in M (min)              | float64  |
| Est Dia in M (max)              | float64  |
| Est Dia in Miles (min)          | float64  |
| Est Dia in Miles (max)          | float64  |
| Est Dia in Feet (min)           | float64  |
| Est Dia in Feet (max)           | float64  |
| Close Approach Date             | object   |
| Epoch Date Close Approach       | int64    |
| Relative Velocity km per sec    | float64  |
| Relative Velocity km per hr     | float64  |
| Miles per hour                  | float64  |
| Miss Dist. (Astronomical)       | float64  |
| Miss Dist. (lunar)              | float64  |
| Miss Dist. (kilometers)         | float64  |
| Miss Dist. (miles)              | float64  |
| Orbiting Body                   | object   |
| Orbit ID                        | int64    |
| Orbit Determination Date        | object   |
| Orbit Uncertainty               | int64    |
| Minimum Orbit Intersection      | float64  |
| Jupiter Tisserand Invariant     | float64  |
| Epoch Osculation                | float64  |
| Eccentricity                    | float64  |
| Semi Major Axis                 | float64  |
| Inclination                     | float64  |
| Asc Node Longitude              | float64  |
| Orbital Period                  | float64  |
| Perihelion Distance             | float64  |
| Perihelion Arg                  | float64  |
| Aphelion Dist                   | float64  |
| Perihelion Time                 | float64  |
| Mean Anomaly                    | float64  |
| Mean Motion                     | float64  |
| Equinox                         | object   |
| Hazardous                       | bool     |


### Exploratory Data Analysis

In [ ]:
df = pd.read_csv('data/nasa.csv')
df.head()

By taking a look at the table entries, it was clear that the dataset was unbalanced.

In [ ]:
df['Hazardous'].value_counts()

df['Hazardous'].value_counts().plot(kind='pie', autopct='%1.1f%%', colors=['red', 'green'])
plt.ylabel('')
plt.title('Asteroid Hazard Classification')
plt.show()

In [ ]:
df.info()

Looking at the data, we found columns that were not needed, since they were essentially duplicates with different units of measurement.
We also removed identification columns since they didn't provide any useful information for classifying the dataset.

In [ ]:
cols_to_drop = [
    # Remove duplicated columns (same data, different units of measure)
    "Est Dia in KM(min)",
    "Est Dia in KM(max)",
    "Est Dia in Miles(min)",
    "Est Dia in Miles(max)",
    "Est Dia in Feet(min)",
    "Est Dia in Feet(max)",
    "Relative Velocity km per hr",
    "Miles per hour",
    "Miss Dist.(Astronomical)",
    "Miss Dist.(lunar)",
    "Miss Dist.(miles)",

    # Remove identification columns
    "Neo Reference ID",
    "Name",
    "Orbit ID",
    'Close Approach Date',
    'Orbit Determination Date',
]

df.drop(cols_to_drop, axis=1, inplace=True)

We also checked the dataset for any missing or duplicated values and removed data that was not relevant to the problem (columns like ***Equinox*** have a single value for every entry).

In [ ]:
# Check for missing values
df.isna().sum()

In [ ]:
# Check for duplicated values
df.duplicated().sum()

In [ ]:
# Encode target variable
label_encoder = LabelEncoder()
df["Hazardous"] = label_encoder.fit_transform(df["Hazardous"])

df.head()

In [ ]:
# Remove categorical data that is not relevant to the problem (has a single value)
categorical = df.select_dtypes(include="object").columns.tolist()

unique_categorical = [cat for cat in categorical if df[cat].nunique() == 1]
unique_categorical

In [ ]:
df.drop(columns=['Orbiting Body', 'Equinox'], axis=1, inplace=True)

Having done the preprocessing, we then proceeded to analyze the data.

In [ ]:
# Correlation matrix

correlation_matrix = df.corr()
plt.figure(figsize=(15, 10))
sb.heatmap(correlation_matrix, annot=True)
plt.show()

With the heatmap in mind we can see that:

    - The columns `Est Dia in M(min)` and `Est Dia in M(max)` are highly correlated.
    - The columns `Jupiter Tisserand Invariant` and `Mean Motion` are highly correlated.
    - The Columns `Epoch Osculation` and `Perihelion Time` are highly correlated.
    - The columns `Semi Major Axis` and `Orbital Period` are highly correlated.
    - The columns `Semi Major Axis` and `Aphelion Dist` are highly correlated.

Therefore, we removed these columns from the dataset.

In [ ]:
# remove highly correlated columns
cols_to_remove = [
    'Est Dia in M(min)',
    'Jupiter Tisserand Invariant',
    'Epoch Osculation',
    'Orbital Period',
    'Aphelion Dist',
]

df.drop(cols_to_remove, axis=1, inplace=True)
df.info()

In [ ]:
# Select numerical columns
numerical = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
numerical.remove('Hazardous')

numerical

In [ ]:
# Plot boxplots for numerical columns

fig, axes = plt.subplots(math.ceil(len(numerical) / 4), 4, figsize=(25, 25))
fig.subplots_adjust(hspace=0.5, wspace=0.5)
axes = axes.ravel()

# for i, col in enumerate(numerical):
#     sb.boxplot(x='Hazardous', y=col, data=df, ax=axes[i])

for col, axis in zip(numerical, axes):
    sb.boxplot(data=df[col], ax=axis)

for i in range(len(numerical), len(axes)):
    fig.delaxes(axes[i])

plt.show()

After removing the outliers, we used the ```describe()``` method to get a better understanding of the data we're about to utilize:

In [ ]:
# Remove outliers

for col in numerical:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

df.describe()

## Creating the Training and Testing Data

Having processed and analyzed the data, we then created the training and testing datasets. We decided to utilize a 75/25 split - 75% of the dataset is used for training and the remaining 25% is used for testing.

In [ ]:
new_data = df.drop('Hazardous', axis=1)
hazard = df['Hazardous']

(training_inputs,
     testing_inputs,
     training_classes,
     testing_classes) = train_test_split(new_data, hazard, test_size=0.25, random_state=1)

## Model Training

We also created a ```data_results``` function to help us analyze and compare the results for each of the classifier algorithms.

In [ ]:
def data_results(testing_classes, testing_inputs, alg_class):
    cm_display = ConfusionMatrixDisplay(
        confusion_matrix=confusion_matrix(testing_classes, alg_class.predict(testing_inputs))
    )

    cm_display.plot()
    plt.xticks([0, 1], ["False", "True"])
    plt.yticks([0, 1], ["False", "True"])
    plt.xlabel('Predicted Hazard')
    plt.ylabel('Actual Hazard')
    plt.show()

    print(classification_report(testing_classes, alg_class.predict(testing_inputs)))

***Note***: *We utilized some default/hardcoded values for the parameters in some algorithms, like SVM or ANN; it is, however, perfectly possible to use different values for these.*

### Decision Tree

In [ ]:
dt_class = DecisionTreeClassifier(random_state=1)
dt_class.fit(training_inputs, training_classes)

dt_class.score(testing_inputs, testing_classes)

accuracy_score(testing_classes, dt_class.predict(testing_inputs))

data_results(testing_classes, testing_inputs, dt_class)

### Nearest Neighbors

In [ ]:
knn_class = neighbors.KNeighborsClassifier(n_neighbors=5)
knn_class.fit(training_inputs, training_classes)

knn_class.score(testing_inputs, testing_classes)

accuracy_score(testing_classes, knn_class.predict(testing_inputs))

data_results(testing_classes, testing_inputs, knn_class)

### Support Vector Machine

In [ ]:
svm_class = SVC(kernel='rbf')
svm_class.fit(training_inputs, training_classes)

svm_class.score(testing_inputs, testing_classes)

accuracy_score(testing_classes, svm_class.predict(testing_inputs))

data_results(testing_classes, testing_inputs, svm_class)

### Neural Network

In [ ]:
ann_class = MLPClassifier(hidden_layer_sizes=(25*4, 25*2, 25), activation='logistic', solver='adam',
                            max_iter=1000, random_state=1)
ann_class.fit(training_inputs, training_classes)

ann_class.score(testing_inputs, testing_classes)

accuracy_score(testing_classes, ann_class.predict(testing_inputs))

data_results(testing_classes, testing_inputs, ann_class)

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb_class = GaussianNB()
nb_class.fit(training_inputs, training_classes)

nb_class.score(testing_inputs, testing_classes)

accuracy_score(testing_classes, nb_class.predict(testing_inputs))

data_results(testing_classes, testing_inputs, nb_class)

### Random Forest

In [ ]:
rf_class = RandomForestClassifier(n_estimators=100)
rf_class.fit(training_inputs, training_classes)

rf_class.score(testing_inputs, testing_classes)

accuracy_score(testing_classes, rf_class.predict(testing_inputs))

data_results(testing_classes, testing_inputs, rf_class)

## Conclusion

It was clear that the Decision Tree and Random Forest algorithms were, by far, the best models for this problem.

Due to the nature of the dataset - small and unbalanced - we found that Nearest Neighbors, Support Vector Machines, Neural Networks and Naive Bayes were not suitabe for this problem.